# <font color="red">**WEEKLY INFLATION FORECAST: Fine-tuning Hyperparameters**</font>

**Author:** Osmar Bolivar

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.graph_objects as go

In [2]:
def lagged_correlation_color(df, specific_var, l=0):
    lags = [l]
    # Create lagged versions of the DataFrame
    lagged_df = pd.concat([df.shift(lag) for lag in lags], axis=1, keys=[f'Lag{lag}' for lag in lags])
    # Calculate correlation coefficients
    correlations = lagged_df.corrwith(df[specific_var])
    # Sort correlations from highest to lowest
    correlations_sorted = correlations.sort_values(ascending=False)
    # Convert the sorted correlations to a data frame and reset index
    df_correlations_sorted = pd.DataFrame(correlations_sorted, columns=['corr']).reset_index()

    lagged_df.columns = df.columns
    return df_correlations_sorted, lagged_df

col_range = range(0,1)

## **1. Monthly data**

In [3]:
dataset_m = pd.read_csv("./Data/M_DATASET.csv", index_col=0)
dataset_m['exchange'] = 1/dataset_m['exchange']  ## to ensure a positive correlation
dataset_m

,ipc_all,lag_1,lag_2,lag_3,lag_6,lag_9,lag_12,beef_lp,beef_cb,beef_sc,...,inflacion,inflacion en bolivia,inflación bolivia,ipc,la inflacion,la inflación,pib,pib bolivia,que es inflacion,que es pib
2011-01-31,74.207255,73.260267,71.989803,71.196381,69.071086,68.561311,68.467691,18.492308,20.980769,18.450000,...,58,59,100,79,44,33,50,29,100,54
2011-02-28,75.439060,74.207255,73.260267,71.989803,69.800954,68.549203,68.581371,18.720833,20.500000,18.385000,...,63,44,100,79,45,33,59,51,100,54
2011-03-31,76.108818,75.439060,74.207255,73.260267,70.335479,68.646613,68.499278,19.166667,20.518519,18.140370,...,82,78,100,54,57,33,92,77,100,54
2011-04-30,76.125495,76.108818,75.439060,74.207255,71.196381,69.071086,68.561311,19.076923,19.153846,17.835769,...,100,90,100,93,100,33,98,77,100,59
2011-05-31,76.277495,76.125495,76.108818,75.439060,71.989803,69.800954,68.549203,19.000000,18.884615,17.827692,...,75,68,100,88,73,33,78,69,89,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-30,110.425657,110.429431,110.440281,110.506839,109.439785,108.697854,108.692935,23.211200,23.000000,21.000000,...,41,39,59,54,53,44,47,37,42,38
2023-12-31,111.123491,110.425657,110.429431,110.440281,109.678594,108.614602,108.818364,23.223333,23.000000,21.000000,...,31,39,68,67,55,51,35,24,54,44
2024-01-31,111.211060,111.123491,110.425657,110.429431,110.081702,108.814654,109.176930,24.120909,23.965909,20.954545,...,29,38,64,86,43,46,30,27,53,43
2024-02-29,111.433348,111.211060,111.123491,110.425657,110.506839,109.439785,108.697854,23.987391,24.956522,21.000000,...,31,38,58,66,38,38,37,34,51,41


In [4]:
ldf_corr = dataset_m.copy()
ldf_corr = ldf_corr.dropna(axis = 1)

g12df_corr = ldf_corr.copy().pct_change(12).dropna(axis=0)
g12df_corr = g12df_corr.loc[:, np.isfinite(g12df_corr).all(axis=0)]
g12df_corr = g12df_corr.dropna(axis = 1)

g1df_corr = ldf_corr.copy().pct_change(1).dropna(axis=0)
g1df_corr = g1df_corr.loc[:, np.isfinite(g1df_corr).all(axis=0)]
g1df_corr = g1df_corr.dropna(axis = 1)

In [5]:
ldf_corr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159 entries, 2011-01-31 to 2024-03-31
Columns: 492 entries, ipc_all to que es pib
dtypes: float64(475), int64(17)
memory usage: 612.4+ KB


In [6]:
g12df_corr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 147 entries, 2012-01-31 to 2024-03-31
Columns: 492 entries, ipc_all to que es pib
dtypes: float64(492)
memory usage: 566.2+ KB


In [7]:
g1df_corr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 158 entries, 2011-02-28 to 2024-03-31
Columns: 492 entries, ipc_all to que es pib
dtypes: float64(492)
memory usage: 608.5+ KB


In [8]:
lcorr_lag0, ldf_lag0 = lagged_correlation_color(ldf_corr, 'ipc_all', 0)
lcorr_lag1, ldf_lag1 = lagged_correlation_color(ldf_corr, 'ipc_all', 1)
lcorr_lag2, ldf_lag2 = lagged_correlation_color(ldf_corr, 'ipc_all', 2)
lcorr_lag3, ldf_lag3 = lagged_correlation_color(ldf_corr, 'ipc_all', 3)

g12corr_lag0, g12df_lag0 = lagged_correlation_color(g12df_corr, 'ipc_all', 0)
g12corr_lag1, g12df_lag1 = lagged_correlation_color(g12df_corr, 'ipc_all', 1)
g12corr_lag2, g12df_lag2 = lagged_correlation_color(g12df_corr, 'ipc_all', 2)
g12corr_lag3, g12df_lag3 = lagged_correlation_color(g12df_corr, 'ipc_all', 3)

g1corr_lag0, g1df_lag0 = lagged_correlation_color(g1df_corr, 'ipc_all', 0)
g1corr_lag1, g1df_lag1 = lagged_correlation_color(g1df_corr, 'ipc_all', 1)
g1corr_lag2, g1df_lag2 = lagged_correlation_color(g1df_corr, 'ipc_all', 2)
g1corr_lag3, g1df_lag3 = lagged_correlation_color(g1df_corr, 'ipc_all', 3)

c:\Users\osmar.bolivar\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\osmar.bolivar\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\osmar.bolivar\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\osmar.bolivar\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\osmar.bolivar\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\osmar.bolivar\AppData\Local\Programs\Python\Py

In [9]:
lcorr_lag0.query('corr > 0.5').style.background_gradient(cmap='coolwarm', low=0.5, high=1)

,level_0,level_1,corr
0,Lag0,ipc_all,1.000000
1,Lag0,lag_1,0.999413
2,Lag0,lag_2,0.998665
3,Lag0,lag_3,0.997993
4,Lag0,ufv,0.997520
5,Lag0,lag_6,0.997022
6,Lag0,lag_9,0.995984
7,Lag0,lag_12,0.995014
8,Lag0,milk_lp,0.912873
9,Lag0,milk_or,0.902819


In [10]:
lcorr_lag1.query('corr > 0.7').style.background_gradient(cmap='coolwarm', low=0.5, high=1)

,level_0,level_1,corr
0,Lag1,ipc_all,0.999402
1,Lag1,lag_1,0.998674
2,Lag1,lag_2,0.998004
3,Lag1,lag_3,0.997549
4,Lag1,ufv,0.997132
5,Lag1,lag_6,0.996674
6,Lag1,lag_9,0.995513
7,Lag1,lag_12,0.994651
8,Lag1,milk_lp,0.912360
9,Lag1,milk_or,0.902378


In [11]:
lcorr_lag2.query('corr > 0.7').style.background_gradient(cmap='coolwarm', low=0.5, high=1)

,level_0,level_1,corr
0,Lag2,ipc_all,0.998690
1,Lag2,lag_1,0.998081
2,Lag2,lag_2,0.997650
3,Lag2,lag_3,0.997451
4,Lag2,ufv,0.996844
5,Lag2,lag_6,0.996329
6,Lag2,lag_9,0.995073
7,Lag2,lag_12,0.994495
8,Lag2,milk_lp,0.911571
9,Lag2,milk_or,0.901260


In [12]:
lcorr_lag3.query('corr > 0.7').style.background_gradient(cmap='coolwarm', low=0.5, high=1)

,level_0,level_1,corr
0,Lag3,ipc_all,0.998108
1,Lag3,lag_1,0.997761
2,Lag3,lag_2,0.997598
3,Lag3,lag_3,0.997379
4,Lag3,ufv,0.996591
5,Lag3,lag_6,0.996088
6,Lag3,lag_9,0.994677
7,Lag3,lag_12,0.994324
8,Lag3,milk_lp,0.911136
9,Lag3,milk_or,0.900377


In [13]:
g12corr_lag0.query('corr > 0.5').style.background_gradient(cmap='coolwarm', low=0.5, high=1)

,level_0,level_1,corr
0,Lag0,ipc_all,1.000000
1,Lag0,lag_1,0.951487
2,Lag0,lag_2,0.877528
3,Lag0,lag_3,0.812737
4,Lag0,ufv,0.768418
5,Lag0,lag_6,0.762980
6,Lag0,lag_9,0.694489
7,Lag0,quinoa_sc,0.596201
8,Lag0,lag_12,0.581161
9,Lag0,milk2_tr,0.577394


In [14]:
g1corr_lag0.query('corr > 0.3').style.background_gradient(cmap='coolwarm', low=0.5, high=1)

,level_0,level_1,corr
0,Lag0,ipc_all,1.000000
1,Lag0,tomato_lp,0.481644
2,Lag0,tomato_bol,0.460381
3,Lag0,tomato_cb,0.452370
4,Lag0,tomato_po,0.449583
5,Lag0,tomato_su,0.447627
6,Lag0,papa2_or,0.408961
7,Lag0,chicken_lp,0.394342
8,Lag0,tomato_or,0.380713
9,Lag0,beef_or,0.369734


In [17]:
df = ldf_corr.copy()

In [15]:
np.array(lcorr_lag0.query('corr > 0.5')['level_1'])

array(['ipc_all', 'lag_1', 'lag_2', 'lag_3', 'ufv', 'lag_6', 'lag_9',
       'lag_12', 'milk_lp', 'milk_or', 'milk_bol', 'paprika_tr',
       'milk_sc', 'milk_su', 'beef_lp', 'milk_po', 'squash_tr',
       'banana_co', 'milk_cb', 'corn_co', 'papaya_tr', 'milk2_po',
       'milk2_or', 'apple_sc', 'wheat_sc', 'beef_su', 'beef_bol',
       'rice3_co', 'onion2_po', 'sorghum_lp', 'redpepper_tr', 'flour_tj',
       'banana_bol', 'rice2_co', 'apple_or', 'rice_co', 'banana_tj',
       'beef_or', 'dinero', 'flour_po', 'beef_sc', 'grapefruit_po',
       'bean_tr', 'greenbean_tr', 'beef_cb', 'libor', 'banana_tr',
       'rice4_or', 'zinc', 'grapefruit_bol', 'banana_sc', 'pineapple_or',
       'grapefruit_cb', 'redpepper_po', 'banana_su', 'wheat_po',
       'milk2_lp', 'veglard_co', 'banana_lp', 'peas_tr', 'beef_tr',
       'grapefruit_or', 'oil_co', 'rice2_or', 'soy_po', 'grapefruit_su',
       'platano_tr', 'beef_po'], dtype=object)

In [21]:
predictors1 = ['ipc_nal', 'ipc_nal_l1', 'ipc_nal_l2', 'ipc_nal_l3', 'ipc_nal_l6', 'ipc_nal_l12',
               'ufv', 'milk_lp', 'milk_dlp', 'milk_or', 'milk_bol', 'paprika_tr', 'milk_sc', 'rice3_ea_y', 'rice3_ea_x', 'beef_dlp',
               'beef_lp', 'milk_po', 'milk_su', 'squash_tr', 'milk_cb', 'banana_co', 'banana_ea', 'pineapple_ea', 'Precio - Tema',
               'beef_bol', 'corn_co', 'milk2_or', 'Interés - Tema', 'milk2_po', 'ycorn_ea', 'papaya_tr', 'milk_ea', 'wheat_sc', 'apple_sc',
               'dinero', 'Dinero - Tema', 'beef_su', 'sorghum_lp', 'onion2_po', 'rice_ea', 'banana_bol', 'redpepper_tr', 'Política - Tema',
               'banana_tj', 'flour_tj', 'platano_ea', 'milk2_dlp', 'la inflación', 'sorghum_bol', 'Inflación', 'beef_or', 'flour_po',
               'sorghum_dlp', 'exchange', 'grapefruit_po', 'beef_sc', 'bean_tr', 'corn_ea', 'rice2_co', 'zinc', 'Inflación - Tema', 'apple_or', 'flour2_ea',
               'banana_tr', 'beef_cb', 'milk2_lp', 'greenbean_tr', 'libor', 'banana_sc', 'peas_tr', 'apple_bol', 'grapefruit_cb',
               'pineapple_or', 'que es inflación', 'wheat_bol', 'redpepper_po', 'peas_ea', 'papa1_ea', 'banana_su', 'lemon_cb', 'rice2_or',
               'banana_lp', 'quinoa_ea', 'Salario - Tema', 'oil_co', 'beef_tr']


df = df[predictors1]

## **2. Train and Val sets**

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [23]:
train, validation = train_test_split(df, test_size=0.2, random_state=42)
print(f'Obs in train set: {train.shape[0]}; variables in train set: {train.shape[1]}')
print(f'Obs in validation set: {validation.shape[0]}; variables in validation set: {validation.shape[1]}')

Obs in train set: 124; variables in train set: 87
Obs in validation set: 32; variables in validation set: 87


In [24]:
scaler = StandardScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(train), columns=train.columns, index=train.index)
validation_scaled = pd.DataFrame(scaler.transform(validation), columns=validation.columns, index=validation.index)
#test_scaled = pd.DataFrame(scaler.transform(test), columns=test.columns, index=test.index)

X_train = train_scaled.drop('ipc_nal', axis=1)
y_train = train_scaled['ipc_nal']

X_validation = validation_scaled.drop('ipc_nal', axis=1)
y_validation = validation_scaled['ipc_nal']

#X_test = test_scaled.drop('ipc_nal', axis=1)

## **3. Algorithms**

In [25]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

tscv5 = TimeSeriesSplit(n_splits=5)

### **3.1. Ridge**
Without tuning:
Validation MSE:  0.001453348422371209
Validation R2:  0.9984692589865253
Validation MAE:  0.030880852442606015

In [33]:
#ridge = Ridge()
ridge = Ridge(alpha=0.11326825671361537, fit_intercept=False, positive=True, random_state=0)  ## cv=5
# Fit on training set
ridge.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
ridge_train_pred = ridge.predict(X_train)
ridge_val_pred = ridge.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_ridge = mean_squared_error(y_train, ridge_train_pred, squared=True)
r2_train_ridge = r2_score(y_train, ridge_train_pred)
mae_train_ridge = mean_absolute_error(y_train, ridge_train_pred)
print("Train MSE: ", mse_train_ridge)
print("Train R2: ", r2_train_ridge)
print("Train MAE: ", mae_train_ridge)
# Calculate Forecast metrics on validation set
mse_val_ridge = mean_squared_error(y_validation, ridge_val_pred, squared=True)
r2_val_ridge = r2_score(y_validation, ridge_val_pred)
mae_val_ridge = mean_absolute_error(y_validation, ridge_val_pred)
print("Validation MSE: ", mse_val_ridge)
print("Validation R2: ", r2_val_ridge)
print("Validation MAE: ", mae_val_ridge)

Train MSE:  0.0009685506142711325
Train R2:  0.9990314493857289
Train MAE:  0.02399079998055779
Validation MSE:  0.0008789962502787182
Validation R2:  0.9990741961182323
Validation MAE:  0.02468902048744895


In [27]:
# Define parameter grid for GridSearchCV
alphas_ridge = np.logspace(-1, 2, num=500)
#alphas_ridge = np.arange(1, 100, 0.05)
param_grid = {'alpha': alphas_ridge,
              'positive': [True, False],
              'fit_intercept': [True, False]}

# Instantiate Ridge model
ridge = Ridge()

# Define GridSearchCV object
grid_search_ridge = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')  ## works better with 5 cv
#grid_search_ridge = GridSearchCV(ridge, param_grid, cv=5, scoring='r2')  ## works better with 5 cv

# Fit GridSearchCV on training set
grid_search_ridge.fit(X_train, y_train)

# Print best parameter and score from GridSearchCV
print("Best parameter: ", grid_search_ridge.best_params_)
print("Best score: ", -grid_search_ridge.best_score_)

# Make predictions on validation set using best model from GridSearchCV
best_ridge = grid_search_ridge.best_estimator_
y_val_pred = best_ridge.predict(X_validation)

# Calculate RMSE on validation set
mse_val = mean_squared_error(y_validation, y_val_pred, squared=True)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Make predictions on test set using best model from GridSearchCV
#y_test_pred_ridge = best_ridge.predict(X_test)

Best parameter:  {'alpha': 0.11017528137883871, 'fit_intercept': False, 'positive': True}
Best score:  0.0025758252589013564
Validation MSE:  0.0008932617208701132
Validation R2:  0.9990591709937855
Validation MAE:  0.024832503304094394


In [ ]:
# Get the coefficients from the Ridge model
coef = ridge.coef_
# Create a dataframe of feature importances
feature_importance_ridge = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_ridge = feature_importance_ridge.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_ridge.columns = ['feat_ridge', 'imp_ridge']
# Print the feature importances
feature_importance_ridge.head(15)

,feat_ridge,imp_ridge
0,ipc_nal_l1,0.171552
1,ipc_nal_l2,0.140606
2,ufv,0.119632
3,ipc_nal_l3,0.117031
4,ipc_nal_l6,0.115594
5,ipc_nal_l12,0.107948
6,rice2_or,0.030317
7,Precio - Tema,0.025489
8,greenbean_tr,0.020595
9,milk_po,0.018265


### **3.2. Lasso**
Without tuning:
Validation MSE:  0.9523622169433499
Validation R2:  -0.0030766763969758415
Validation MAE:  0.8455503308136401

In [ ]:
#lasso = Lasso()
lasso = Lasso(alpha=0.1, fit_intercept=False, positive=True)  ## cv=5
# Fit on training set
lasso.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
lasso_train_pred = lasso.predict(X_train)
lasso_val_pred = lasso.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_lasso = mean_squared_error(y_train, lasso_train_pred, squared=True)
r2_train_lasso = r2_score(y_train, lasso_train_pred)
mae_train_lasso = mean_absolute_error(y_train, lasso_train_pred)
print("Train MSE: ", mse_train_lasso)
print("Train R2: ", r2_train_lasso)
print("Train MAE: ", mae_train_lasso)
# Calculate Forecast metrics on validation set
mse_val_lasso = mean_squared_error(y_validation, lasso_val_pred, squared=True)
r2_val_lasso = r2_score(y_validation, lasso_val_pred)
mae_val_lasso = mean_absolute_error(y_validation, lasso_val_pred)
print("Validation MSE: ", mse_val_lasso)
print("Validation R2: ", r2_val_lasso)
print("Validation MAE: ", mae_val_lasso)

Train MSE:  0.01135096248401723
Train R2:  0.9886490375159828
Train MAE:  0.0886406245973438
Validation MSE:  0.009937567975745763
Validation R2:  0.9895332443063803
Validation MAE:  0.08505641169339459


In [ ]:
# Define parameter grid for GridSearchCV
alphas_lasso = np.logspace(-1, 2, num=500)
#alphas_lasso = np.arange(1, 100, 0.05)
param_grid = {'alpha': alphas_lasso,
              'positive': [True, False],
              'fit_intercept': [True, False]}

# Instantiate lasso model
lasso = Lasso()

# Define GridSearchCV object
grid_search_lasso = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_mean_squared_error')  ## works better with 5 cv
#grid_search_lasso = GridSearchCV(lasso, param_grid, cv=5, scoring='r2')  ## works better with 5 cv

# Fit GridSearchCV on training set
grid_search_lasso.fit(X_train, y_train)

# Print best parameter and score from GridSearchCV
print("Best parameter: ", grid_search_lasso.best_params_)
print("Best score: ", -grid_search_lasso.best_score_)

# Make predictions on validation set using best model from GridSearchCV
best_lasso = grid_search_lasso.best_estimator_
y_val_pred = best_lasso.predict(X_validation)

# Calculate RMSE on validation set
mse_val = mean_squared_error(y_validation, y_val_pred, squared=True)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Make predictions on test set using best model from GridSearchCV
#y_test_pred_lasso = best_lasso.predict(X_test)

Best parameter:  {'alpha': 0.1, 'fit_intercept': False, 'positive': True}
Best score:  0.012353938460066747
Validation MSE:  0.009937567975745763
Validation R2:  0.9895332443063803
Validation MAE:  0.08505641169339459


In [ ]:
# Get the coefficients from the Lasso model
coef = lasso.coef_
# Create a dataframe of feature importances
feature_importance_lasso = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_lasso = feature_importance_lasso.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_lasso.columns = ['feat_lasso', 'imp_lasso']
# Print the feature importances
feature_importance_lasso.head(15)

,feat_lasso,imp_lasso
0,ipc_nal_l1,0.899295
1,ipc_nal_l6,0.000053
2,beef_sc,0.000000
3,banana_tr,0.000000
4,flour2_ea,0.000000
5,apple_or,0.000000
6,Inflación - Tema,0.000000
7,zinc,0.000000
8,rice2_co,0.000000
9,corn_ea,0.000000


### **3.3. ElasticNet**
Without tuning:
Validation MSE:  0.2878895501474107
Validation R2:  0.6967799772043428
Validation MAE:  0.4602160948451647

In [ ]:
#enet = ElasticNet()
enet = ElasticNet(alpha=0.11326825671361537, l1_ratio=0.1, fit_intercept=False, positive=True)  ## cv=5
# Fit on training set
enet.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
enet_train_pred = enet.predict(X_train)
enet_val_pred = enet.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_enet = mean_squared_error(y_train, enet_train_pred, squared=True)
r2_train_enet = r2_score(y_train, enet_train_pred)
mae_train_enet = mean_absolute_error(y_train, enet_train_pred)
print("Train MSE: ", mse_train_enet)
print("Train R2: ", r2_train_enet)
print("Train MAE: ", mae_train_enet)
# Calculate Forecast metrics on validation set
mse_val_enet = mean_squared_error(y_validation, enet_val_pred, squared=True)
r2_val_enet = r2_score(y_validation, enet_val_pred)
mae_val_enet = mean_absolute_error(y_validation, enet_val_pred)
print("Validation MSE: ", mse_val_enet)
print("Validation R2: ", r2_val_enet)
print("Validation MAE: ", mae_val_enet)

Train MSE:  0.0023597541727734526
Train R2:  0.9976402458272265
Train MAE:  0.03813004212820444
Validation MSE:  0.002107268648489714
Validation R2:  0.9977805167040469
Validation MAE:  0.03561740379378739


In [ ]:
# Define parameter grid for GridSearchCV
alphas_enet = np.logspace(-1, 2, num=500)
#alphas_enet = np.arange(1, 100, 0.05)
param_grid = {'alpha': alphas_enet,
              'l1_ratio': np.arange(0.01, 0.95, 0.01),
              'positive': [True, False],
              'fit_intercept': [True, False]}

# Instantiate enet model
enet = ElasticNet()

# Define GridSearchCV object
grid_search_enet = GridSearchCV(enet, param_grid, cv=5, scoring='neg_mean_squared_error')  ## works better with 5 cv
#grid_search_enet = GridSearchCV(enet, param_grid, cv=5, scoring='r2')  ## works better with 5 cv

# Fit GridSearchCV on training set
grid_search_enet.fit(X_train, y_train)

# Print best parameter and score from GridSearchCV
print("Best parameter: ", grid_search_enet.best_params_)
print("Best score: ", -grid_search_enet.best_score_)

# Make predictions on validation set using best model from GridSearchCV
best_enet = grid_search_enet.best_estimator_
y_val_pred = best_enet.predict(X_validation)

# Calculate RMSE on validation set
mse_val = mean_squared_error(y_validation, y_val_pred, squared=True)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Make predictions on test set using best model from GridSearchCV
#y_test_pred_enet = best_enet.predict(X_test)

In [ ]:
# Get the coefficients from the enet model
coef = enet.coef_
# Create a dataframe of feature importances
feature_importance_enet = pd.DataFrame({'Feature': X_train.columns, 'Importance': coef})
# Sort the features by importance
feature_importance_enet = feature_importance_enet.sort_values('Importance', ascending=False).reset_index(drop=True)
feature_importance_enet.columns = ['feat_enet', 'imp_enet']
# Print the feature importances
feature_importance_enet.head(15)

,feat_enet,imp_enet
0,ipc_nal_l1,0.086441
1,ipc_nal_l2,0.083880
2,ipc_nal_l3,0.081695
3,ufv,0.080271
4,ipc_nal_l6,0.079604
5,ipc_nal_l12,0.076850
6,paprika_tr,0.036142
7,Precio - Tema,0.033811
8,rice2_or,0.032323
9,beef_dlp,0.024307


### **3.4. ADA**
Without tuning:
Validation MSE:  0.0036814337026600556
Validation R2:  0.9961225254245258
Validation MAE:  0.0469554086526466

In [ ]:
#ada = AdaBoostRegressor()
ada = AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=5), n_estimators=115, learning_rate=1.235482888256747, random_state=0)
# Fit the model on training data
ada.fit(X_train, y_train)
# Make predictions on the validation set
ada_train_pred = ada.predict(X_train)
ada_val_pred = ada.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_ada = mean_squared_error(y_train, ada_train_pred, squared=True)
r2_train_ada = r2_score(y_train, ada_train_pred)
mae_train_ada = mean_absolute_error(y_train, ada_train_pred)
print("Train MSE: ", mse_train_ada)
print("Train R2: ", r2_train_ada)
print("Train MAE: ", mae_train_ada)
# Evaluate the model on the validation set
mse_val_ada = mean_squared_error(y_validation, ada_val_pred, squared=True)
r2_val_ada = r2_score(y_validation, ada_val_pred)
mae_val_ada = mean_absolute_error(y_validation, ada_val_pred)
print("Validation MSE: ", mse_val_ada)
print("Validation R2: ", r2_val_ada)
print("Validation MAE: ", mae_val_ada)

Train MSE:  0.0001518067359025188
Train R2:  0.9998481932640975
Train MAE:  0.008301392934766337
Validation MSE:  0.0014618738110729937
Validation R2:  0.998460279610389
Validation MAE:  0.02910552415848012


In [ ]:
# Define the AdaBoost Regressor
ada = AdaBoostRegressor()

# Define the range of hyperparameters to search over
param_grid_ada = {
    'n_estimators': range(50, 200, 5),    ##120 was selected range(50, 200, 5)
    'learning_rate': np.logspace(-2,0.5, 50),
    #'loss': ['linear', 'square', 'exponential']
    'random_state': [0],
    'estimator': [DecisionTreeRegressor(max_depth=3),
                  DecisionTreeRegressor(max_depth=4),
                  DecisionTreeRegressor(max_depth=5),
                  DecisionTreeRegressor(max_depth=6),
                  ]
}

# Tune hyperparameters using GridSearchCV with TimeSeriesSplit
#grid_search_ada = GridSearchCV(estimator=ada, param_grid=param_grid_ada, cv=5, scoring='r2')
grid_search_ada = GridSearchCV(estimator=ada, param_grid=param_grid_ada, cv=5, scoring='neg_mean_squared_error')
grid_search_ada.fit(X_train, y_train)

# Evaluate the model using the best hyperparameters on the test set
ada_best = AdaBoostRegressor(**grid_search_ada.best_params_)
ada_best.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred = ada_best.predict(X_validation)

# Evaluate the model on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred, squared=True)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Print the best hyperparameters and the best score
print("Best parameters found: ", grid_search_ada.best_params_)
print("Lowest MSE found: ", -grid_search_ada.best_score_)

# Evaluate the model on the test set
#y_test_pred_ada = ada_best.predict(X_test)

Validation MSE:  0.0014618738110729937
Validation R2:  0.998460279610389
Validation MAE:  0.02910552415848012
Best parameters found:  {'estimator': DecisionTreeRegressor(max_depth=5), 'learning_rate': 1.235482888256747, 'n_estimators': 115, 'random_state': 0}
Lowest MSE found:  -0.9981856980134719


In [ ]:
# Create a DataFrame with the feature importance values
feature_importance_ada = pd.DataFrame({'Feature': X_train.columns, 'Importance': ada.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_ada = feature_importance_ada.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_ada.columns = ['feat', 'imp_ada']
# Print the feature importance DataFrame
feature_importance_ada.head(15)

,feat,imp_ada
0,ipc_nal_l3,0.090550
1,ufv,0.084265
2,ipc_nal_l1,0.071661
3,ipc_nal_l2,0.066128
4,ipc_nal_l6,0.062670
5,flour_tj,0.062154
6,ipc_nal_l12,0.045471
7,milk2_or,0.036227
8,milk2_lp,0.036167
9,milk2_po,0.033045


### **3.5. GBR**
Without tuning:
Validation MSE:  0.0014618738110729937
Validation R2:  0.998460279610389
Validation MAE:  0.02910552415848012

In [ ]:
#gbr = GradientBoostingRegressor(random_state=0)
gbr = GradientBoostingRegressor(learning_rate=0.040949150623804255, n_estimators=290, random_state=0)
# Fit the model on the training set
gbr.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
gbr_train_pred = gbr.predict(X_train)
gbr_val_pred = gbr.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_gbr = mean_squared_error(y_train, gbr_train_pred, squared=True)
r2_train_gbr = r2_score(y_train, gbr_train_pred)
mae_train_gbr = mean_absolute_error(y_train, gbr_train_pred)
print("Train MSE: ", mse_train_gbr)
print("Train R2: ", r2_train_gbr)
print("Train MAE: ", mae_train_gbr)
# Calculate Forecast metrics on validation set
mse_val_gbr = mean_squared_error(y_validation, gbr_val_pred, squared=True)
r2_val_gbr = r2_score(y_validation, gbr_val_pred)
mae_val_gbr = mean_absolute_error(y_validation, gbr_val_pred)
print("Validation MSE: ", mse_val_gbr)
print("Validation R2: ", r2_val_gbr)
print("Validation MAE: ", mae_val_gbr)

Train MSE:  2.4216058773004873e-06
Train R2:  0.9999975783941227
Train MAE:  0.0011680068652647226
Validation MSE:  0.0009299617979484944
Validation R2:  0.9990205165924618
Validation MAE:  0.02345452785681064


In [ ]:
# Define the model
gbr = GradientBoostingRegressor()

# Define the hyperparameters to be tuned
params = {
    'learning_rate': np.logspace(-2,0.5, 50),
    #'loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],   ## 'loss': 'squared_error'
    'n_estimators': range(100, 300, 10),
    'max_depth': range(3,6,1),
    #'min_samples_split': range(2,20,1),
    #'min_samples_leaf': range(1,50, 1),
    #'min_weight_fraction_leaf': np.arange(0.0, 0.5, 0.01),
    #'subsample': np.arange(0.5, 1, 0.05),
    #'max_features': [None, 'sqrt', 'log2'],
    #'max_leaf_nodes': range(2, 200, 1),
    #'criterion': ['friedman_mse', 'squared_error'],
    'random_state': [0]
}

# Create the GridSearchCV object
#grid_gbr = GridSearchCV(gbr, params, cv=5, scoring='r2', n_jobs=-1)
grid_gbr = GridSearchCV(gbr, params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the model on the training set with GridSearchCV
grid_gbr.fit(X_train, y_train)

# Print the best hyperparameters
print('Best hyperparameters:', grid_gbr.best_params_)

# Use the best model to make predictions on the validation set
y_val_pred = grid_gbr.predict(X_validation)

# Compute the mean squared error of the predictions on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred, squared=True)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Use the best model to make predictions on the test set
#y_test_pred_gbr = grid_gbr.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:961: RuntimeWarning:

overflow encountered in square



Best hyperparameters: {'learning_rate': 0.040949150623804255, 'max_depth': 3, 'n_estimators': 290, 'random_state': 0}
Validation MSE:  0.0009299617979484944
Validation R2:  0.9990205165924618
Validation MAE:  0.02345452785681064


In [ ]:
# Create a DataFrame with the feature importance values
feature_importance_gbr = pd.DataFrame({'Feature': X_train.columns, 'Importance': gbr.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_gbr = feature_importance_gbr.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_gbr.columns = ['feat_gbr', 'imp_gbr']
# Print the feature importance DataFrame
feature_importance_gbr.head(15)

,feat_gbr,imp_gbr
0,ipc_nal_l6,0.181419
1,ufv,0.161772
2,ipc_nal_l3,0.130876
3,ipc_nal_l1,0.121839
4,ipc_nal_l12,0.118458
5,flour_tj,0.099594
6,ipc_nal_l2,0.065873
7,ycorn_ea,0.053459
8,paprika_tr,0.010656
9,milk_or,0.008036


### **3.4. RF**
Without tuning:
Validation MSE:  0.00100147205985504
Validation R2:  0.9989451983211514
Validation MAE:  0.026535714623290185

In [ ]:
# Define the Random Forest Regression model
#rf = RandomForestRegressor(random_state=0)
rf = RandomForestRegressor(min_samples_split=2, n_estimators=265, random_state=0)

# Fit the model to the training data and make predictions on the validation set
rf.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
rf_train_pred = rf.predict(X_train)
rf_val_pred = rf.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_rf = mean_squared_error(y_train, rf_train_pred, squared=True)
r2_train_rf = r2_score(y_train, rf_train_pred)
mae_train_rf = mean_absolute_error(y_train, rf_train_pred)
print("Train MSE: ", mse_train_rf)
print("Train R2: ", r2_train_rf)
print("Train MAE: ", mae_train_rf)
# Calculate Forecast metrics on validation set
mse_val_rf = mean_squared_error(y_validation, rf_val_pred, squared=True)
r2_val_rf = r2_score(y_validation, rf_val_pred)
mae_val_rf = mean_absolute_error(y_validation, rf_val_pred)
print("Validation MSE: ", mse_val_rf)
print("Validation R2: ", r2_val_rf)
print("Validation MAE: ", mae_val_rf)

Train MSE:  0.0003599513395981111
Train R2:  0.9996400486604019
Train MAE:  0.01178117103932864
Validation MSE:  0.0009983900502684609
Validation R2:  0.9989484444515393
Validation MAE:  0.024970310604635504


In [ ]:
# Define the Random Forest Regression model
rf_reg = RandomForestRegressor()

# Define the hyperparameters to tune
param_grid_rf = {
    'n_estimators': range(100, 300, 15),
    #'max_features': [None, 'sqrt', 'sqrt']
    #'max_depth': range(3,7,1),
    'min_samples_split': range(2,20,1),
    #'min_samples_leaf': range(1,50, 1),
    #'min_weight_fraction_leaf': np.arange(0.0, 0.5, 0.01),
    #'bootstrap': [True],
    #'oob_score': [True, False],
    #'warm_start': [True, False],
    #'max_samples': np.arange(0.1, 1.0, 0.01)
    'random_state': [0]
}

# Define the GridSearchCV object
#grid_rf_reg = GridSearchCV(estimator=rf_reg, param_grid=param_grid_rf, cv=5, scoring='r2')
grid_rf_reg = GridSearchCV(estimator=rf_reg, param_grid=param_grid_rf, cv=5, scoring='neg_mean_squared_error')

# Fit the GridSearchCV object to the training data
grid_rf_reg.fit(X_train, y_train)

# Extract the best hyperparameters and score
best_params = grid_rf_reg.best_params_
best_score = grid_rf_reg.best_score_

# Print the best hyperparameters found by GridSearchCV
print(f"Best hyperparameters: {best_params}")
print(f"Best score: {best_score}")

# Instantiate a new Random Forest Regression model using the best hyperparameters
rf_reg_best = RandomForestRegressor(**best_params)

# Fit the model to the training data and make predictions on the validation set
rf_reg_best.fit(X_train, y_train)

# Use the best model to make predictions on the validation set
y_val_pred = rf_reg_best.predict(X_validation)

# Compute the mean squared error of the predictions on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred, squared=True)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Use the best model to make predictions on the test set
#y_test_pred_rf = rf_reg_best.predict(X_test)

Best hyperparameters: {'min_samples_split': 2, 'n_estimators': 265, 'random_state': 0}
Best score: -0.0035817383690906874
Validation MSE:  0.0009983900502684609
Validation R2:  0.9989484444515393
Validation MAE:  0.024970310604635504


In [ ]:
# Create a DataFrame with the feature importance values
feature_importance_rf = pd.DataFrame({'Feature': X_train.columns, 'Importance': rf.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_rf = feature_importance_rf.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_rf.columns = ['feat', 'imp_rf']
# Print the feature importance DataFrame
feature_importance_rf.head(30)

,feat,imp_rf
0,ipc_nal_l6,0.137714
1,ipc_nal_l2,0.118994
2,ufv,0.117774
3,ipc_nal_l12,0.110887
4,ipc_nal_l1,0.104852
5,ipc_nal_l3,0.094636
6,flour_tj,0.090228
7,milk2_po,0.023239
8,milk2_or,0.023221
9,milk_bol,0.015986


### **3.5. ET**

In [ ]:
# Define the Extra Trees Regression model
et = ExtraTreesRegressor()
#et = ExtraTreesRegressor(bootstrap=True, max_samples=0.9599999999999995, oob_score=True)
# Fit the model to the training data and make predictions on the validation set
et.fit(X_train, y_train)
# Make predictions on validation set using best model from GridSearchCV
et_train_pred = et.predict(X_train)
et_val_pred = et.predict(X_validation)
# Calculate Forecast metrics on train set
mse_train_et = mean_squared_error(y_train, et_train_pred, squared=True)
r2_train_et = r2_score(y_train, et_train_pred)
mae_train_et = mean_absolute_error(y_train, et_train_pred)
print("Train MSE: ", mse_train_et)
print("Train R2: ", r2_train_et)
print("Train MAE: ", mae_train_et)
# Calculate Forecast metrics on validation set
mse_val_et = mean_squared_error(y_validation, et_val_pred, squared=True)
r2_val_et = r2_score(y_validation, et_val_pred)
mae_val_et = mean_absolute_error(y_validation, et_val_pred)
print("Validation MSE: ", mse_val_et)
print("Validation R2: ", r2_val_et)
print("Validation MAE: ", mae_val_et)

Train MSE:  2.0896434057490927e-30
Train R2:  1.0
Train MAE:  1.0328599534787715e-15
Validation MSE:  0.0004652877455378555
Validation R2:  0.9995099351097103
Validation MAE:  0.016635965301522112


In [ ]:
# Define the Extra Trees Regression model
et_reg = ExtraTreesRegressor()

# Define the hyperparameter grid to search over
param_grid = {
    #'n_estimators': range(100, 300, 15),
    #'max_depth': range(3,15,1),
    #'max_features': [None, 'sqrt', 'sqrt']
    'min_samples_split': range(2,20,1),
    'min_samples_leaf': range(1,50, 1),
    #'min_weight_fraction_leaf': np.arange(0.0, 0.5, 0.01),
    'bootstrap': [True],
    'oob_score': [True, False],
    #'warm_start': [True, False],
    'max_samples': np.arange(0.1, 1.0, 0.01)
    #'criterion': ['squared_error', 'absolute_error', 'friedman_mse'],
    #'min_impurity_decrease': np.arange(0.0, 0.01, 0.00001),
    #'random_state': [0]
}

# Use GridSearchCV to find the best hyperparameters
#grid_search = GridSearchCV(et_reg, param_grid=param_grid, cv=5, scoring='r2')
grid_search = GridSearchCV(et_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding validation score
print("Best hyperparameters: ", grid_search.best_params_)
#print("Validation score: ", grid_search.best_score_)

# Use the best model to make predictions on the validation set
best_et_reg = grid_search.best_estimator_
y_val_pred = best_et_reg.predict(X_validation)

# Compute the mean squared error of the predictions on the validation set
mse_val = mean_squared_error(y_validation, y_val_pred, squared=True)
r2_val = r2_score(y_validation, y_val_pred)
mae_val = mean_absolute_error(y_validation, y_val_pred)
print("Validation MSE: ", mse_val)
print("Validation R2: ", r2_val)
print("Validation MAE: ", mae_val)

# Use the best model to make predictions on the test set
#y_test_pred_et = best_et_reg.predict(X_test)

In [ ]:
# Create a DataFrame with the feature importance values
feature_importance_et = pd.DataFrame({'Feature': X_train.columns, 'Importance': et.feature_importances_})
# Sort the DataFrame by importance values in descending order
feature_importance_et = feature_importance_et.sort_values(by='Importance', ascending=False).reset_index(drop=True)
feature_importance_et.columns = ['feat', 'imp_et']
# Print the feature importance DataFrame
feature_importance_et.head(30)

,feat,imp_et
0,ycorn_ea,0.128451
1,ipc_nal_l1,0.115000
2,ufv,0.081138
3,ipc_nal_l2,0.073717
4,ipc_nal_l12,0.073355
5,flour_tj,0.068439
6,ipc_nal_l6,0.058653
7,milk_dlp,0.053411
8,milk_lp,0.047272
9,milk_bol,0.045265


## **4. Report**

#### End